In [1]:
from datetime import date, timedelta
import os
import pickle
import sys
sys.path.append("..")

In [2]:
from abautomator import experiment

In [3]:
import pandas as pd


In [4]:
# %pip install -U pandas

In [5]:
name = "rel_diff_ci"
df = pickle.load(
    open(os.path.join("..", "tests", f"{name}.p"), "rb" )
)

In [6]:
df.head(2)

,exp_cond,metric,tx_mean,tx_std,tx_count,factor_label,ctrl_mean,ctrl_std,ctrl_count,abs_mean,abs_std,upper_68_ci,lower_68_ci,upper_95_ci,lower_95_ci,mean,std
0,Video01,n_user_sessions,2.097734,2.643838,3530.0,"(n_user_sessions, Video01)",2.085166,2.522356,3546.0,0.012567,0.061436,3.549024,-2.343621,6.495346,-5.289944,0.602701,2.946323
1,Video01,pct_user_sessions,0.798584,0.401115,3530.0,"(pct_user_sessions, Video01)",0.823463,0.381330,3546.0,-0.024879,0.009305,-1.891318,-4.151330,-0.761312,-5.281336,-3.021324,1.130006


In [113]:
import itertools
import math

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import CDSView, ColumnDataSource, IndexFilter, FactorRange, BooleanFilter
from bokeh.models import BoxZoomTool, ResetTool, PanTool
from bokeh.palettes import Colorblind8
from bokeh.transform import factor_cmap


from math import pi

output_notebook()

Loading BokehJS ...

In [169]:
def add_error_bars(fig_core):
    upper_eb = _get_error_bar("upper_68_ci", "upper_95_ci", fig_core)
    lower_eb = _get_error_bar("lower_95_ci", "lower_68_ci", fig_core)
    
    return lower_eb, upper_eb
    
    
def _get_error_bar(left_label, right_label, fig_core):
    plot, source, view = fig_core
    return plot.segment(
        right_label,
        "factor_label",
        left_label,
        "factor_label",
        color="black",
        source=source,
        view=view,
    )

def add_core_interval(exp_cond, color, fig_core):
    plot, source, view = fig_core
    return p.hbar(
        y='factor_label',
        right="upper_68_ci",
        left="lower_68_ci",
        legend_label=exp_cond,
        fill_color=color,
        line_color=None,
        height=0.6,
        fill_alpha=0.8,
        source=source,
        view=view,
    )

def _get_tool_tips():
    return [
        ("Rel Pop. Diff. x̄", "@{mean}±@{std}"),
        ("Abs Pop. Diff. x̄", "@{abs_mean}±@{abs_std}"),
        ("Tx x̄", "@{tx_mean}±@{tx_std}"),
        ("Ctrl x̄", "@{ctrl_mean}±@{ctrl_std}"),
        ("Tx Label", "@{exp_cond}"),
        ("Ctrl/Tx n", "@{ctrl_count}/@{tx_count}"),
    ]

In [170]:
def _set_legend(plot):
    plot.legend.location = "top_right"
    plot.legend.click_policy="hide"

def _set_x_axis(plot):
    plot.xaxis.axis_label = 'Relative difference from control (%)'

def _set_y_axis(plot):
    plot.yaxis.major_label_text_alpha = 0.0
    plot.yaxis.major_tick_in = 0
    plot.yaxis.major_tick_out = 0
    plot.yaxis.major_label_text_font_size = '1px'
    
    plot.yaxis.group_label_orientation = "horizontal"
    plot.yaxis.separator_line_alpha = 0
    plot.yaxis.minor_tick_line_color = "yellow"
    plot.ygrid.grid_line_color = None

In [171]:
source = ColumnDataSource(df)
view = CDSView(source=source, filters=[IndexFilter([0, 2, 4])])

p = figure(
    y_range=FactorRange(*list(source.data["factor_label"])),
    height=450,
    title=f"exp name",
    toolbar_location="right",
    tools=[BoxZoomTool(), ResetTool(), PanTool()],
    tooltips= _get_tool_tips(),
)

colors = itertools.cycle(Colorblind8)

for exp_cond, color in zip(df["exp_cond"].unique(), colors):
    bools = [True if exp_cond == filter_cond else False for filter_cond in source.data['exp_cond']]
    view = CDSView(source=source, filters=[BooleanFilter(bools)])
    
    fig_core = (p, source, view)
    
    lower_eb, upper_eb = add_error_bars(fig_core)
    core_interval = add_core_interval(exp_cond, color, fig_core)

    core_interval.js_link('visible', lower_eb, 'visible')
    core_interval.js_link('visible', upper_eb, 'visible')

_set_legend(p)
_set_x_axis(p)
_set_y_axis(p)


In [172]:
show(p)

In [56]:
type(hbar)

bokeh.models.renderers.GlyphRenderer